In [1]:
import os
import sys
from pathlib import Path
import re
import jieba
from lib.io import parse_file_to_dict, write_dict_to_file, get_new_id, copy_file_if_exists

## 准备词典

In [2]:
# !cd data && python ../tools/prep_dict.py lexicon.txt

In [3]:
# !./tools/prepare_dict.sh data

In [4]:
# !rm -rf gmm/data/dict/
# !mkdir gmm/data/dict/
# !cd data &&cp lexicon.txt silence_phones.txt nonsilence_phones.txt extra_questions.txt optional_silence.txt ../gmm/data/dict
# # !rm silence_phones.txt nonsilence_phones.txt extra_questions.txt optional_silence.txt

## 16k生成wav.scp

In [2]:
dir_16k = "/home1/meichaoyang/Dataset/16k"
wav_list_16k=list(Path(dir_16k).rglob("*.wav"))

In [ ]:
corp_raw_16k = "/home1/meichaoyang/Dataset/16k/usort-c.txt"
corp_map_16k = {}
with open(corp_raw_16k, "r") as f:
    for line in f:
        data = line.split()
        corp_map_16k[data[0].split("/")[-1].split(".")[0]] = data[1]

In [ ]:
wav_list_16k.sort(key=lambda Path: os.path.basename(os.path.splitext((str(Path)))[0]))

In [8]:
scp_16k = {}
for path_item in wav_list_16k:
    wav_path = str(path_item)
    scp_16k[os.path.basename(os.path.splitext((wav_path))[0])] = wav_path

In [9]:
with open('tmp/16k_wav.scp', 'w') as f:
    for i in sorted(scp_16k):
        if i in corp_map_16k:
            f.write(i+'\t'+scp_16k[i]+'\n')

In [10]:
with open('tmp/16k_trans.txt', 'w') as f:
    for i in sorted(corp_map_16k):
        if i in scp_16k:
            f.write(i+'\t'+corp_map_16k[i]+'\n')

## Aishell2数据处理

### 生成Aishell2的原始wav.scp

In [13]:
aishell_dir = "/home1/meichaoyang/Dataset/data_aishell2/data_aishell/wav/train"
aishell_wav_list=list(Path(aishell_dir).rglob("*.wav"))
aishell_wav_list.sort(key=lambda Path: os.path.basename(os.path.splitext((str(Path)))[0]))

In [14]:
aishell_scp_raw = {}
with open('tmp/Aishell_wav_raw.scp', 'w') as f:
    for path_item in aishell_wav_list:
        wav_path = str(path_item)
        aishell_scp_raw[os.path.basename(os.path.splitext((wav_path))[0])] = wav_path
        f.write(os.path.basename(os.path.splitext((wav_path))[0])+'\t'+wav_path+'\n')

In [16]:
aishell_2_corp_raw = "/home1/meichaoyang/Dataset/data_aishell2/data_aishell/wav/trans.txt"
pattern_Eng = re.compile(u'[a-zA-Z\n]')
corp_map = {}
with open(aishell_2_corp_raw, "r") as f:
    for line in f:
        data = line.split()
        if pattern_Eng.search(data[1]) != None: ##删除小于10和非英文标注
            continue
        corp_map[data[0].split("/")[-1].split(".")[0]] = data[1]

In [17]:
aishell_scp = {}
for i in sorted(aishell_scp_raw):
    if i not in corp_map.keys():
        continue
    aishell_scp[i] = aishell_scp_raw[i]

In [18]:
with open('tmp/aishell_2.scp', 'w') as f:
    for i in sorted(aishell_scp):
        if i in corp_map:
            f.write(i+'\t'+aishell_scp[i]+'\n')

In [19]:
with open('tmp/aishell_2_trans.txt', 'w') as f:
    for i in sorted(corp_map):
        if i in aishell_scp:
            f.write(i+'\t'+corp_map[i]+'\n')

## 合并数据

In [2]:
wav_scp_16k = {}
with open("tmp/16k_wav.scp", "r") as f:
    for line in f:
        data = line.split()
        wav_scp_16k[data[0]] = data[1]
        
wav_corp_16k = {}      
with open("tmp/16k_trans.txt", "r") as f:
    for line in f:
        data = line.split()
        if data[0] in wav_scp_16k:
            wav_corp_16k[data[0]] = "".join(data[1:])

In [3]:
aishell_wav_scp = parse_file_to_dict("tmp/aishell_2.scp")        
aishell_wav_corp = parse_file_to_dict("tmp/aishell_2_trans.txt")     

In [4]:
wav_scp = {**wav_scp_16k,**aishell_wav_scp}
corpus = {**wav_corp_16k, **aishell_wav_corp}

In [5]:
print(len(wav_scp),len(corpus))

2384349 2384349


In [6]:
with open('data/wav.scp', 'w') as f:
    for i in sorted(wav_scp):
        f.write(i+'\t'+wav_scp[i]+'\n')

In [7]:
with open('data/text', 'w') as f:
    for i in sorted(corpus):
        f.write(i+'\t'+corpus[i]+'\n')

### 生成词典并分词

In [21]:
#!python tools/filter_scp.py corpus.seg wav.scp wav.scp.new

## utt2spk和spk2utt

In [7]:
new_wav_scp = parse_file_to_dict("data/wav.scp",assert2fields=True)

In [8]:
utts = list(new_wav_scp.keys())

In [9]:
utt2spk = {}
for utt in utts:
    if utt.startswith("A"):
        spk = utt
    else:
        spk = utt.split("W")[0]
    utt2spk[utt] = spk

In [10]:
spk2utt = {}
for utt in utt2spk:
    spk = utt2spk[utt]
    if spk not in spk2utt:
        spk2utt[spk] = []
    spk2utt[spk].append(utt)

In [36]:
with open('data/utt2spk', 'w') as f:
    for i in sorted(utt2spk):
        f.write(i+'\t'+utt2spk[i]+'\n')

In [34]:
len(utts)

2384349

#### 根据utt2spk生成spk2utt

In [37]:
!gmm/utils/utt2spk_to_spk2utt.pl data/utt2spk > data/spk2utt

## 数据增强

In [ ]:
!cd ~/Dataset/audio_aug && bash wav_aug_common.sh --aug-list "reverb babble volume" --srcdir /home1/meichaoyang/workspace/git/16k_model/aishell_16k/data

## 整合增强后的数据

In [26]:
for file in ("utt2spk" , "text"):
    for typ in ("reverb","babble", "volume"):
        fields = [0,1]
        if file == "text":
            fields = [0]
        copy_file_if_exists("data/"+file, "data_"+typ+"/"+file, "prefix", typ,fields)

In [14]:
!cd gmm && bash utils/data/combine_data.sh ../combine_data ../data_reverb ../data_babble ../data_volume

utils/data/combine_data.sh ../combine_data ../data_reverb ../data_babble ../data_volume
utils/data/combine_data.sh: combined utt2uniq
utils/data/combine_data.sh [info]: not combining segments as it does not exist
utils/data/combine_data.sh: combined utt2spk
utils/data/combine_data.sh [info]: not combining utt2lang as it does not exist
utils/data/combine_data.sh [info]: not combining utt2dur as it does not exist
utils/data/combine_data.sh [info]: **not combining reco2dur as it does not exist everywhere**
utils/data/combine_data.sh [info]: not combining feats.scp as it does not exist
utils/data/combine_data.sh: combined text
utils/data/combine_data.sh [info]: not combining cmvn.scp as it does not exist
utils/data/combine_data.sh [info]: not combining vad.scp as it does not exist
utils/data/combine_data.sh [info]: not combining reco2file_and_channel as it does not exist
utils/data/combine_data.sh: combined wav.scp
utils/data/combine_data.sh [info]: not combining spk2gender as it does not 

In [27]:
summed = {}
for file in ("utt2spk" , "text"):
    for typ in ("reverb","babble", "volume"):
        summed = {**summed,**parse_file_to_dict("data_"+typ+"/"+file)}
    write_dict_to_file(summed, "combine_data/" +file)

In [ ]:
!gmm/utils/utt2spk_to_spk2utt.pl combine_data/utt2spk > combine_data/spk2utt

## 分词

In [21]:
!sed 's/\s.*$/ 1/' gmm/data/dict/lexicon.txt > seg.dict

In [22]:
#!python seg_word/segmentword.py seg.dict corpus.txt corpus.seg oov_file


vocab_file="seg.dict"
trans_file="combine_data/text"
word_segmented_trans="combine_data/text.seg"

jieba.set_dictionary(vocab_file)
with open(word_segmented_trans, "w") as f:
    for line in open(trans_file):
      key,trans = line.strip().split(None, 1)
      words = jieba.cut(trans, HMM=False) # turn off new word discovery (HMM-based)
      new_line = key + '\t' + " ".join(words)
      f.write(new_line + "\n")



Building prefix dict from /home1/meichaoyang/workspace/git/16k_model/aishell_16k/seg.dict ...
Dumping model to file cache /home1/meichaoyang/anaconda3/lib/python3.7/site-packages/jieba/cache/jieba.u74b91bd8b6f08f8d6543d6f6267dfa9e.cache
Loading model cost 0.432 seconds.
Prefix dict has been built succesfully.


#### spk2utt，utt2spk，text(corpus.seg)，wav.scp(wav.scp.new)复制到train_{mfcc,fbank}

In [23]:
!rm -rf train_mfcc train_fbank
!mkdir train_mfcc train_fbank
!cd combine_data && cp spk2utt utt2spk text.seg wav.scp ../train_mfcc

In [24]:
!mv train_mfcc/text.seg train_mfcc/text 
#!mv train_mfcc/wav.scp.new train_mfcc/wav.scp
!cd train_mfcc &&cp spk2utt utt2spk text wav.scp ../train_fbank

#### 特征提取`cd ext_feats && sh ext_{mfcc,fbank}.sh {train_fbank,train_mfcc,dev}`，需注意修改conf下mfcc，fbank参数

In [25]:
!cd ext_feats && sh ext_mfcc.sh ../train_mfcc && utils/fix_data_dir.sh ../train_mfcc

steps/make_mfcc.sh --nj 48 --cmd run.pl ../train_mfcc ../train_mfcc/log ../train_mfcc/_mfcc
sort: write failed: 'standard output': Broken pipe
sort: write error
utils/validate_data_dir.sh: utt2spk is not in sorted order when sorted first on speaker-id 
(fix this by making speaker-ids prefixes of utt-ids)


In [ ]:
!cd ext_feats && sh ext_fbank.sh ../train_fbank && utils/fix_data_dir.sh ../train_fbank

In [ ]:
!ln -s ../../train_mfcc gmm/data/train  > /dev/null 2>&1
!ln -s ../../train_fbank chain/data/train_fbank  > /dev/null 2>&1
!ln -s ../../gmm/data/train/ chain/data/train_mfcc  > /dev/null 2>&1

In [ ]:
!cd gmm && make prepare_lang

In [ ]:
!cd gmm &&make -f Makefile gmm  | tee gmm.log

### chain训练

In [ ]:
!export CUDA_VISIBLE_DEVICES=1,3 && cd chain && make chain |  tee chain.log

### 构建解码图

In [ ]:
!cd chain && local/make_graph.sh /home1/meichaoyang/dataset/lm/all.lm.order4.1e-9 data/lang exp/chain/tdnn_attend data/dict

### 解码测试

In [ ]:
!cd chain && bash steps/nnet3/decode.sh --nj 8 exp/chain/tdnn_attend/graph /home1/meichaoyang/dataset/data_aishell2/feats/test aishell_chain_result_dir

# chain模型再训练

### 根据chain模型生成对齐和词图（可更改chain模型下的frame采样因子来改变对齐数据）

In [2]:
!cd chain &&./steps/nnet3/align_lats.sh --generate-ali-from-lats true --nj 24 --scale_opts '--transition-scale=1.0 --self-loop-scale=1.0' --acoustic_scale 1.0 --generate_ali_from_lats true ../train_fbank ../gmm/data/lang exp/chain/tdnn_attend exp/chain/chain_align_lat

./steps/nnet3/align.sh --nj 3 --use_gpu true ../train_fbank ../gmm/data/lang exp/chain/tdnn_attend chain_align
./steps/nnet3/align.sh: no such file exp/chain/tdnn_attend/final.mdl


### 根据chain模型的对齐再训练

In [ ]:
!export CUDA_VISIBLE_DEVICES=3 &&cd chain && bash local/train_chain_again.sh --ali_dir exp/chain/chain_align_lat --alignment_subsampling_factor 1 exp/chain/tdnn_attend_again | tee chain_again.log

### 使用新的声学模型构建解码图

In [ ]:
!cd chain && local/make_graph.sh /home1/meichaoyang/dataset/lm/all.lm.order4.1e-9 data/lang exp/chain/tdnn_attend_again data/dict

### 解码测试新训练的chain模型

In [1]:
!cd chain && bash steps/nnet3/decode.sh --nj 8 exp/chain/tdnn_attend_again/graph /home1/meichaoyang/dataset/data_aishell2/feats/test aishell_chain_again——result_dir

./steps/get_train_ctm.sh data/train_fbank data/lang chain_align
./steps/get_train_ctm.sh: expecting file chain_align/final.mdl to exist


In [47]:
#!sed 's/[^\s]*\s//' corpus.seg > corpus_lm.txt
!ngram-count -text corpus_lm.txt -order 1 -unk -map-unk "<UNK>" -interpolate -lm corpus.lm

In [ ]:
!cd chain/ && make make_graph